# Lab 4 – Pivot Tables, Permutation Testing, and Missing Values

## DSC 80, Fall 2022

### Due Date: Monday, October 24th at 11:59 PM

## Instructions
Much like in DSC 10, this Jupyter Notebook contains the statements of the problems and provides code and Markdown cells to display your answers to the problems. Unlike DSC 10, the notebook is *only* for displaying a readable version of your final answers. The coding will be done in an accompanying `lab.py` file that is imported into the current notebook.

Labs and programming assignments will be graded in (at most) two ways:
1. The functions and classes in the accompanying `lab.py` file will be tested (a la DSC 20),
2. The notebook may be graded (if it contains free response questions or asks you to draw plots).

**Do not change the function names in the `*lab.py` file!**
- The functions in the `lab.py` file are how your assignment is graded, and they are graded by their name.
- If you changed something you weren't supposed to, just use git to revert! Ask us if you need help with this, or google around for `git revert`.

**Tips for working in the notebook**:
- The notebooks serve to present the questions and give you a place to present your results for later review.
- The notebooks in *lab assignments* are not graded (only the `lab.py` file is submitted and graded).
- The notebook serves as a nice environment for 'pre-development' and experimentation before designing your function in your `lab.py` file. You can write code here, but make sure that all of your real work is in the `lab.py` file.

**Tips for developing in the `lab.py` file**:
- Do not change the function names in the starter code; grading is done using these function names.
- Do not change the docstrings in the functions. These are there to tell you if your work is on the right track!
- You are encouraged to write your own additional helper functions to solve the lab! 
- Always document your code!

### Importing code from `lab.py`

* We import our `lab.py` file that's contained in the same directory as this notebook.
* We use the `autoreload` notebook extension to make changes to our `lab.py` file immediately available in our notebook. Without this extension, we would need to restart the notebook kernel to see any changes to `lab.py` in the notebook.
    - `autoreload` is necessary because, upon import, `lab.py` is compiled to bytecode (in the directory `__pycache__`). Subsequent imports of `lab` merely import the existing compiled python.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# from lab import *

In [3]:
import pandas as pd
import numpy as np
import io
import os
from datetime import time

In [4]:
import pandas as pd
import numpy as np
import io
import os

### Question 0 – Mid-Quarter Survey 🙋

Course staff, along with the Data Science Student Representatives, have put together a mid-quarter survey that will allow you to share feedback on your experience in DSC 80 so far. It is **entirely anonymous**, so we encourage you to be honest.

<br>

<center><b><a href=https://forms.gle/5Bd64352PArTH7Z89>Click here to access the mid-quarter survey.</a></b></center>

<br>

We'd like to have as many students as possible in the class fill out the survey. **As such, if 80% of the class fills out this survey before the Midterm Exam, then everyone will earn an extra point on the Midterm Exam.** The survey will close before the Midterm Exam. 

We really appreciate your feedback, thanks! 😊

## Part 1: Time Series Data

Imagine that you own an online store and you'd like to monitor the visits to your site. You've collected some data that you store in `data/login_table.csv`. It contains the information about different login dates and times for different users. Some users are unique, some visited your store multiple times.

You need to answer a few questions below in order to understand the login patters of your users.

### Question 1 – Passwords 🔑

Write a function `latest_login` which takes in a DataFrame like `login` and outputs a DataFrame indexed by `'Login Id'`, counting the number of logins that occurs at the prime time for each user. Prime time is as it says: from 4 PM to 8 PM (inclusive). The DataFrame should have just one column, named `'Time'`.

For example, if a user logs in at 5 PM one day, 1 PM another day, and again at 8 PM, then her total number of prime-time log-ins is 2. Note that the values in your returned DataFrame should only include counts, not datetime objects.

***Note:*** You do not need to use Python's `datetime` module – instead, use the built-in `pandas` methods for working with times that we introduced in [Lecture 6](https://github.com/dsc-courses/dsc80-2022-fa/blob/main/lectures/06-combining/notebook/lecture.ipynb) (though you will need to do a bit more research to fully answer the question). Do not use a `for`-loop.

In [5]:
def latest_login(login):
    """Calculates the latest login time for each user
    :param login: a DataFrame with login information
    :return: a DataFrame with latest login time for
    each user indexed by "Login Id"
    >>> fp = os.path.join('data', 'login_table.csv')
    >>> login = pd.read_csv(fp)
    >>> result = latest_login(login)
    >>> len(result)
    433
    >>> result.loc[381, "Time"] > 7
    True
    """
    # BEGIN SOLUTION
    login_time = login.copy()
    login_time["Time"] = pd.to_datetime(login_time["Time"])
    counts = login_time.groupby('Login Id').agg(lambda ser: sum((ser.dt.hour >= 16) & (ser.dt.hour <= 20)))
    return counts
    # END SOLUTION

In [6]:
# don't change this cell -- it is needed for the tests to work
fp = os.path.join('data', 'login_table.csv')
login = pd.read_csv(fp)
q1_result = latest_login(login)

In [7]:
""" # BEGIN TEST CONFIG
hidden: false
points: .5
""" # END TEST CONFIG
len(q1_result) == 433

True

In [8]:
""" # BEGIN TEST CONFIG
hidden: false
points: .5
""" # END TEST CONFIG
q1_result.loc[381, "Time"] > 7

True

In [9]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message:  You should not use loops in Pandas. Loop was found in latest_login.
""" # END TEST CONFIG
import util
not util.check_loops(latest_login)

True

In [10]:
""" # BEGIN TEST CONFIG
hidden: false
points: .5
failure_message:  'check user 457'
""" # END TEST CONFIG
q1_result.loc[457, 'Time'] == 4

True

In [11]:
""" # BEGIN TEST CONFIG
hidden: false
points: .5
failure_message:  'check user 457'
""" # END TEST CONFIG
q1_result.loc[458, 'Time'] > 50

True

In [12]:
""" # BEGIN TEST CONFIG
hidden: false
points: 1
failure_message:  'check number of unique users'
""" # END TEST CONFIG
len(set(q1_result.index)) == 433

True

In [13]:
""" # BEGIN TEST CONFIG
hidden: true
points: .5
failure_message:  'check user 381'
""" # END TEST CONFIG
q1_result.loc[381, 'Time'] == 9

True

In [14]:
""" # BEGIN TEST CONFIG
hidden: false
points: .5
failure_message:  'check user 381'
""" # END TEST CONFIG
q1_result.loc[381, 'Time'] > 6

True

In [15]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'check the entire list of times'
""" # END TEST CONFIG
np.isclose(q1_result.loc[:, 'Time'].mean(), 2.127, atol=0.5)

True

In [16]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'median hour: approx'
""" # END TEST CONFIG
np.isclose(q1_result.loc[:, 'Time'].median(), 1, atol=2.0)

True

In [17]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'median hour: approx'
""" # END TEST CONFIG
np.isclose(q1_result.loc[:, 'Time'].median(),1, atol=0)

True

In [18]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'check the minimum number of logins'
""" # END TEST CONFIG
q1_result.loc[:, 'Time'].min() == 0

True

In [19]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'check the maximum number of logins'
""" # END TEST CONFIG
q1_result.loc[:, 'Time'].max() == 67

True

### Question 2 – Return Users 🔁

As a site owner, you would like to find your most enthusiastic users -- the ones who return to your site most frequently. You've noticed that there are users who have several logins and users who logged in only once. You are interested in finding the number of logins *per day* for each user.

To do this, you can assume that today is  January 5, 2018. The first login date of users is the first day of their membership in the site, and you can assume that they are still a member today. For simplicity, you only need to count full days that a user has been a member. For example, if a user's first login was 12 days and 5 hours ago, you can say that they have been a user for 12 days. 

Write a function `calculate_frequency` which takes in a DataFrame like `login` and outputs a Series containing the number of logins per day for each user. Your Series should have `'Login Id'`s in its index, and the frequencies as its values. The order of users in the index is arbitrary.

**IMPORTANT**: No Loops Allowed.

***Hint:*** Can you write a custom aggregator that allows you to do this with just one `.groupby`?

In [20]:
def count_frequency(login):
    """
    Calculates the the login frequency for each user.
    :param login: a DataFrame with login information but without unique IDs
    :return: a Series, indexed by Login ID, containing 
    the login frequency for each user.
    >>> fp = os.path.join('data', 'login_table.csv')
    >>> login = pd.read_csv(fp)
    >>> freq = count_frequency(login)
    >>> len(freq)
    433
    >>> np.isclose(freq.loc[466], 0.24517906336088155)
    True
    """
    # BEGIN SOLUTION
    login = login.copy()
    login['Time'] = pd.to_datetime(login['Time'])
    
    def login_frequency(ser):
        days = (pd.Timestamp('2018-01-05 23:59') - ser.min()).days
        # number of logins divided by number of days
        return len(ser) / days

    return login.groupby('Login Id')['Time'].aggregate(login_frequency)
    # END SOLUTION

In [21]:
# don't change this cell -- it is needed for the tests to work
fp = os.path.join('data', 'login_table.csv')
login = pd.read_csv(fp)
q2_result = count_frequency(login)

In [22]:
""" # BEGIN TEST CONFIG
hidden: false
points: .5
""" # END TEST CONFIG
len(q2_result) == 433

True

In [23]:
""" # BEGIN TEST CONFIG
hidden: false
points: .5
""" # END TEST CONFIG
np.isclose(q2_result.loc[466], 0.24517906336088155)

True

In [24]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message:  You should not use loops in Pandas. Loop was found in count_frequency.
""" # END TEST CONFIG
import util
not util.check_loops(count_frequency)

True

In [25]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'max frequency in table'
""" # END TEST CONFIG
q2_result.max() == 2.0

True

In [26]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'number of people whose frequency is greater than 0.5'
""" # END TEST CONFIG
(q2_result > 0.5).sum() == 6

True

In [27]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'number of people whose frequency is less than 0.1'
""" # END TEST CONFIG
(q2_result < 0.1).sum() == 383

True

In [28]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'number of people whose frequency is between 0.2 and 0.4'
""" # END TEST CONFIG
((q2_result > 0.2) & (q2_result < 0.4)).sum() == 12

True

In [29]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'number of people whose frequency is between 0.3 and 0.6'
""" # END TEST CONFIG
((q2_result > 0.3) & (q2_result < 0.6)).sum() == 9

True

In [30]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'min frequency: approx'
""" # END TEST CONFIG
np.isclose(q2_result.min(), 0.0027624309392265192)

True

In [31]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'number of people whose frequency is between 0.7 and 0.8'
""" # END TEST CONFIG
((q2_result > 0.7) & (q2_result < 0.8)).sum() == 0

True

In [32]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'check Login Id 1307'
""" # END TEST CONFIG
q2_result.loc[1307] == 2

True

## Part 2: Pivot Tables

### Question 3 – Summarizing Sales 💰

Recall from [Lecture 5](https://github.com/dsc-courses/dsc80-2022-fa/blob/main/lectures/05-grouping/notebook/lecture.ipynb), a pivot table allows you to aggregate the entries in a DataFrame based on two categorical columns. In this question, you are given a simple dataset, `data/sales.csv`, and are asked to solve a few simple problems using the `pivot_table` method.  

We have provided the outline for your DataFrames, but yours may have a different number of rows and columns and different values.

#### `total_seller`

Write a function `total_seller` that takes in the `sales` DataFrame and returns a DataFrame that contains the total sales for each seller, indexed by `'Name'`. There should not be any `NaN`s.

***Note:*** You may be able to implement `total_seller` without using `pivot_table`.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Total</th>
    </tr>
    <tr>
      <th>Name</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Jones</th>
      <td>0</td>
    </tr>
    <tr>
      <th>Smith</th>
      <td>0</td>
    </tr>
    <tr>
      <th>Trump</th>
      <td>0</td>
    </tr>
  </tbody>
</table>

<br>

#### `product_name`

Write a function `product_name` that takes in the `sales` DataFrame and returns a DataFrame that contains the total sales for each seller, indexed by `'Product'`. Do not fill in `NaN`s.

<table border="1" class="dataframe">
  <thead>
    <tr>
      <th>Name</th>
      <th>Jones</th>
      <th>Smith</th>
      <th>Trump</th>
    </tr>
    <tr>
      <th>Product</th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>boat</th>
      <td>NaN</td>
      <td>NaN</td>
      <td>0.0</td>
    </tr>
    <tr>
      <th>book</th>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
    </tr>
    <tr>
      <th>hotel</th>
      <td>NaN</td>
      <td>NaN</td>
      <td>0.0</td>
    </tr>
    <tr>
      <th>pen</th>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
    </tr>
    <tr>
      <th>ruler</th>
      <td>0.0</td>
      <td>0.0</td>
      <td>NaN</td>
    </tr>
  </tbody>
</table>

<br>

#### `count_product`

Write a function `count_product` that takes in the `sales` DataFrame and returns a DataFrame that contains the total number of items sold product-wise and name-wise per date. Replace `NaN`s with 0s. Don't reset the index after pivoting.

<table border="1" class="dataframe">
  <thead>
    <tr>
      <th></th>
      <th>Date</th>
      <th>01.01.2012</th>
      <th>02.20.2013</th>
      <th>02.25.2015</th>
    </tr>
    <tr>
      <th>Product</th>
      <th>Name</th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>boat</th>
      <th>Trump</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th rowspan="3" valign="top">book</th>
      <th>Jones</th>
      <td>0</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>Smith</th>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>Trump</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>hotel</th>
      <th>Trump</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
  </tbody>
</table>

<br>

#### `total_by_month`

Write a function `total_by_month` that takes in the `sales` DataFrame and returns a pivot table that contains the total sales name-wise, product-wise per month. Replace `NaN`s with 0s. Don't reset the index after pivoting.

<table border="1" class="dataframe">
  <thead>
    <tr>
      <th></th>
      <th>Month</th>
      <th>February</th>
      <th>January</th>
      <th>July</th>
      <th>March</th>
    </tr>
    <tr>
      <th>Name</th>
      <th>Product</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th rowspan="3" valign="top">Jones</th>
      <th>book</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>pen</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>ruler</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th rowspan="3" valign="top">Smith</th>
      <th>book</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>pen</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>ruler</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
  </tbody>
</table>

<br>


***Note:*** [Here](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html) is another great resource that provides an overview of `pivot_table` with many examples from the Titanic dataset.

In [33]:
def total_seller(sales):
    """
    total_seller should take in the sales DataFrame and 
    return a DataFrame that contains the total sales 
    for each seller, indexed by 'Name'. There should not be any NaNs.

    >>> fp = os.path.join('data', 'sales.csv')
    >>> sales = pd.read_csv(fp)
    >>> out = total_seller(sales)
    >>> out.shape[0]
    3
    >>> out["Total"].sum() < 15000
    True
    """
    # BEGIN SOLUTION
    result = sales.pivot_table(index="Name", values="Total", aggfunc='sum')
    return result
    # END SOLUTION


def product_name(sales):
    """
    product_name should take in the sales DataFrame and 
    return a DataFrame that contains the total sales 
    for each seller, indexed by 'Product'. 
    Do not fill in NaNs.
    
    >>> fp = os.path.join('data', 'sales.csv')
    >>> sales = pd.read_csv(fp)
    >>> out = product_name(sales)
    >>> out.size
    15
    >>> out.loc["pen"].isnull().sum()
    0
    """
    # BEGIN SOLUTION
    result = sales.pivot_table(
        index="Product",
        columns="Name",
        values="Total",
        aggfunc='sum'
    )
    return result
    # END SOLUTION


def count_product(sales):
    """
    count_product should take in the sales DataFrame and 
    return a DataFrame that contains the total number of 
    items sold product-wise and name-wise per date. 
    Replace NaNs with 0s.

    >>> fp = os.path.join('data', 'sales.csv')
    >>> sales = pd.read_csv(fp)
    >>> out = count_product(sales)
    >>> out.loc["boat"].loc["Trump"].value_counts()[0]
    6
    >>> out.size
    70
    """
    # BEGIN SOLUTION
    result = sales.pivot_table(
        index=["Product","Name"],
        columns="Date",
        values="Total",
        aggfunc='count',
        fill_value=0
    )
    return result
    # END SOLUTION


def total_by_month(sales):
    """
    total_by_month should take in the sales DataFrame 
    and return a pivot table that contains the total 
    sales name-wise, product-wise per month. 
    Replace NaNs with 0s.
    
    >>> fp = os.path.join('data', 'sales.csv')
    >>> sales = pd.read_csv(fp)
    >>> out = total_by_month(sales)
    >>> out["May"].idxmax()
    ('Smith', 'book')
    >>> out.shape[1]
    5
    """
    # BEGIN SOLUTION
    sales = sales.copy()
    sales['Date']=pd.to_datetime(sales['Date'])
    sales['Month'] = sales['Date'].apply(lambda x:x.month_name())
    result = sales.pivot_table(
        index =["Name","Product"],
        columns="Month",
        values="Total",
        aggfunc='sum',
        fill_value=0
    )
    return result
    # END SOLUTION

In [34]:
# don't change this cell -- it is needed for the tests to work
fp = os.path.join('data', 'sales.csv')
sales = pd.read_csv(fp)
q3_total_seller_out = total_seller(sales)
q3_product_name_out = product_name(sales)
q3_product_count_out = count_product(sales)
q3_total_by_month_out = total_by_month(sales)

In [35]:
""" # BEGIN TEST CONFIG
hidden: false
points: 0.5
""" # END TEST CONFIG
q3_total_seller_out.shape[0] == 3

True

In [36]:
""" # BEGIN TEST CONFIG
hidden: false
points: 0.5
""" # END TEST CONFIG
q3_total_seller_out["Total"].sum() < 15000

True

In [37]:
""" # BEGIN TEST CONFIG
hidden: false
points: 0.5
""" # END TEST CONFIG
q3_product_name_out.size == 15

True

In [38]:
""" # BEGIN TEST CONFIG
hidden: false
points: 0.5
""" # END TEST CONFIG
q3_product_name_out.loc["pen"].isnull().sum() == 0

True

In [39]:
""" # BEGIN TEST CONFIG
hidden: true
points: 0.5
""" # END TEST CONFIG
q3_product_count_out.loc["boat"].loc["Trump"].value_counts()[0] == 6

True

In [40]:
""" # BEGIN TEST CONFIG
hidden: false
points: 0.5
""" # END TEST CONFIG
q3_product_count_out.size == 70

True

In [41]:
""" # BEGIN TEST CONFIG
hidden: true
points: 0.5
""" # END TEST CONFIG
q3_total_by_month_out["May"].idxmax() == ('Smith', 'book')

True

In [42]:
""" # BEGIN TEST CONFIG
hidden: false
points: 0.5
""" # END TEST CONFIG
q3_total_by_month_out.shape[1] == 5

True

In [43]:
""" # BEGIN TEST CONFIG
hidden: true
points: 0.25
failure_message:  You should not use loops in Pandas. Loop was found in total_seller.
""" # END TEST CONFIG
import util
not util.check_loops(total_seller)

True

In [44]:
""" # BEGIN TEST CONFIG
hidden: true
points: 0.25
failure_message:  You should not use loops in Pandas. Loop was found in product_name.
""" # END TEST CONFIG
import util
not util.check_loops(product_name)

True

In [45]:
""" # BEGIN TEST CONFIG
hidden: true
points: 0.25
failure_message:  You should not use loops in Pandas. Loop was found in count_product.
""" # END TEST CONFIG
import util
not util.check_loops(count_product)

True

In [46]:
""" # BEGIN TEST CONFIG
hidden: true
points: 0.25
failure_message:  You should not use loops in Pandas. Loop was found in total_by_month.
""" # END TEST CONFIG
import util
not util.check_loops(total_by_month)

True

In [47]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'Smith Total'
""" # END TEST CONFIG
q3_total_seller_out.loc['Smith', 'Total'] == 6800

True

In [48]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'Jones Total'
""" # END TEST CONFIG
q3_total_seller_out.loc['Jones', 'Total'] == 3680

True

In [49]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'boat / Trump'
""" # END TEST CONFIG
int(q3_product_name_out.loc['boat', 'Trump']) == 700

True

In [50]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'ruler / Smith'
""" # END TEST CONFIG
int(q3_product_name_out.loc['ruler', 'Smith']) == 2100

True

In [51]:
""" # BEGIN TEST CONFIG
hidden: false
points: 1
failure_message: 'null values: hotel row'
""" # END TEST CONFIG
int(q3_product_name_out.loc['hotel'].isnull().sum()) == 2

True

In [52]:
""" # BEGIN TEST CONFIG
hidden: false
points: 1
failure_message: 'correct shape'
""" # END TEST CONFIG
q3_product_count_out.shape == (10, 7)

True

In [53]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'Total number of 0 entries.'
""" # END TEST CONFIG
(q3_product_count_out == 0).sum().sum() == 58

True

In [54]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'books for smith in May'
""" # END TEST CONFIG
int(q3_total_by_month_out.loc[('Smith', 'book'), 'May']) == 2000

True

In [55]:
""" # BEGIN TEST CONFIG
hidden: false
points: .5
failure_message: 'number of entries strictly over 2000'
""" # END TEST CONFIG
(q3_total_by_month_out > 2000).sum().sum() == 3

True

In [56]:
""" # BEGIN TEST CONFIG
hidden: false
points: .5
failure_message: 'number of zero entries'
""" # END TEST CONFIG
(q3_total_by_month_out == 0).sum().sum() == 39

True

## Part 3: Permutation Testing

[Skittles](https://en.wikipedia.org/wiki/Skittles_(confectionery)) 🍬 are made in two locations in the United States: Yorkville, Illinois and Waco, Texas. In these factories, Skittles of different colors are made separately by different machines and combined/packaged into bags for sale. The **tab-separated file** `data/skittles.tsv` contains the contents of 468 bags of Skittles.

Throughout this question, we will compare the color distribution of Skittles between bags made in the Yorkville factory and bags made in the Waco factory. Most people have preferences for their favorite flavor and there is a surprising amount of variation among the distribution of flavors in each bag.

Look at the variation by bag in the dataset below:

In [57]:
skittles_fp = os.path.join('data', 'skittles.tsv')
skittles = pd.read_csv(skittles_fp, sep='\t')
skittles.head()

,red,orange,yellow,green,purple,Factory
0,10,15,11,7,18,Yorkville
1,5,12,17,15,10,Yorkville
2,16,11,15,11,9,Waco
3,15,8,13,16,7,Waco
4,11,14,20,8,7,Waco


In [58]:
skittles.shape

(468, 6)

### Question 4 – Orange Skittles 🟠

First, you will investigate if the machine that mixes together the Skittles of different colors might favor one color over another. Use a permutation test to assess whether, on average, bags made in Yorkville have the same number of orange skittles as bags made in Waco. Do this by implementing the functions described below.

<br>

#### `diff_of_median_proportions`

Create a function `diff_of_median_proportions` that takes in a DataFrame like `skittles` and returns the **absolute difference** between the **median proportion** of orange Skittles per bag from Yorkville and the **median proportion** of orange Skittles per bag from Waco.

<br>

#### `simulate_null`

Create a function `simulate_null` that takes in a DataFrame like `skittles` and returns one simulated instance of the test statistic under the null hypothesis. Note that this will involve shuffling!

<br>

#### `pval_color`

Create a function `pval_color` that takes in a DataFrame like `skittles` and calculates the p-value for the permutation test using 1000 trials.

<br>

Plot the observed statistic, along with the histogram for the simulated distribution, to check your work.

***Note:*** In all functions, the default argument for `col` is `'orange'`. Your functions should still work for any color so that you can call it in later questions.

In [59]:
def diff_of_median_proportions(data, col='orange'):
    """
    diff_of_median_proportions takes in a DataFrame like skittles 
    and returns the absolute difference of median proportions 
    between the number of oranges per bag from Yorkville and Waco.
    :Example:
    >>> skittles_fp = os.path.join('data', 'skittles.tsv')
    >>> skittles = pd.read_csv(skittles_fp, sep='\\t')
    >>> out = diff_of_median_proportions(skittles)
    >>> 0 <= out
    True
    """
    # BEGIN SOLUTION
    data['total_skittles'] = data.sum(axis=1,numeric_only=True) # Finding total number of skittles in the current bag
    data[col+'_proportion'] = data.apply(lambda row : row[col]/row['total_skittles'],axis=1)
    out = data.groupby('Factory')[col+'_proportion'].median().diff().abs().iloc[-1]
    data.drop(col+'_proportion',axis=1,inplace=True)
    data.drop('total_skittles',axis=1,inplace=True)
    return out
    # END SOLUTION


def simulate_null(data, col='orange'):
    """
    simulate_null takes in a DataFrame like skittles and 
    returns one simulated instance of the test statistic 
    under the null hypothesis.
    :Example:
    >>> skittles_fp = os.path.join('data', 'skittles.tsv')
    >>> skittles = pd.read_csv(skittles_fp, sep='\\t')
    >>> out = simulate_null(skittles)
    >>> isinstance(out, float)
    True
    >>> 0 <= out <= 1.0
    True
    """
    # BEGIN SOLUTION
    shuffled = data[col].sample(frac=1, replace=False).reset_index(drop=True)
    dg = data.assign(shuffled=shuffled)
    stat = diff_of_median_proportions(dg, col='shuffled')
    return stat
    # END SOLUTION


def pval_color(data, col='orange'):
    """
    pval_color takes in a DataFrame like skittles and 
    calculates the p-value for the permutation test 
    using 1000 trials.
    
    :Example:
    >>> skittles_fp = os.path.join('data', 'skittles.tsv')
    >>> skittles = pd.read_csv(skittles_fp, sep='\\t')
    >>> pval = pval_color(skittles)
    >>> isinstance(pval, float)
    True
    >>> 0 <= pval <= 0.1
    True
    """
    # BEGIN SOLUTION
    results = []
    for _ in range(1000):
        results.append(simulate_null(data, col))
        
    obs = diff_of_median_proportions(data, col)
    pval = (pd.Series(results) >= obs).mean()
    
    return pval
    # END SOLUTION

In [60]:
# don't change this cell -- it is needed for the tests to work
# cell may take about 1-2 minutes to execute to completion
skittles_fp = os.path.join('data', 'skittles.tsv')
skittles = pd.read_csv(skittles_fp, sep='\\t', engine='python')
q4_diff_of_median_proportions_out = diff_of_median_proportions(skittles)
q4_simulate_null_out = simulate_null(skittles)
q4_many_diffs = np.array([simulate_null(skittles) for _ in range(100)])
q4_pval_out = pval_color(skittles)

In [61]:
""" # BEGIN TEST CONFIG
hidden: false
points: 0.5
""" # END TEST CONFIG
0 <= q4_diff_of_median_proportions_out

True

In [62]:
""" # BEGIN TEST CONFIG
hidden: false
points: 0.5
""" # END TEST CONFIG
isinstance(q4_simulate_null_out, float)

True

In [63]:
""" # BEGIN TEST CONFIG
hidden: false
points: 0.5
""" # END TEST CONFIG
0 <= q4_simulate_null_out <= 1.0

True

In [64]:
""" # BEGIN TEST CONFIG
hidden: false
points: 0.5
""" # END TEST CONFIG
isinstance(q4_pval_out, float)

True

In [65]:
""" # BEGIN TEST CONFIG
hidden: false
points: 0.5
""" # END TEST CONFIG
0 <= q4_pval_out <= 1

True

In [66]:
""" # BEGIN TEST CONFIG
hidden: false
points: 1
failure_message: 'greater than zero'
""" # END TEST CONFIG
q4_diff_of_median_proportions_out > 0

True

In [67]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'answer: approx within 0.01'
""" # END TEST CONFIG
np.isclose(q4_diff_of_median_proportions_out, 0.008705483323911828, atol=0.01)

True

In [68]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'answer: approx within 0.001'
""" # END TEST CONFIG
np.isclose(q4_diff_of_median_proportions_out,0.008705483323911828, atol=0.001)

True

In [69]:
""" # BEGIN TEST CONFIG
hidden: false
points: 1
failure_message: 'greater than zero'
""" # END TEST CONFIG
q4_simulate_null_out >= 0

True

In [70]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'answer: mean null distribution near 0.00004'
""" # END TEST CONFIG
np.isclose(q4_many_diffs.mean(), 0.00482790384428229, atol=0.001)

True

In [71]:
""" # BEGIN TEST CONFIG
hidden: false
points: 1
failure_message: 'orange p-value: doctest'
""" # END TEST CONFIG
0 <= q4_pval_out <= 1

True

In [72]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'orange p-value: within 0.05'
""" # END TEST CONFIG
np.isclose(q4_pval_out, 0.113, atol=0.05)

True

In [73]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'orange p-value: approx: within 0.075'
""" # END TEST CONFIG
np.isclose(q4_pval_out, 0.113, atol=0.075)

True

In [74]:
""" # BEGIN TEST CONFIG
hidden: false
points: 1
failure_message: 'orange p-value: approx'
""" # END TEST CONFIG
np.isclose(q4_pval_out, 0.113, atol=0.1)

True

### Question 5 – Generalizing to all colors 🔴🟠🟡🟢🟣

While your `pval_color` function used a default color of `'orange'`, it should also work for all other colors of Skittles, meaning you can run the same permutation test from Question 4 on all colors of Skittles. Call `pval_color` on all colors of Skittles to find which colors differ the most between the two locations on average. 

Then, create a function `ordered_colors` that returns a list of five ordered pairs, each of the form `('color', p_value)`. For example, your list might look like `[('pink', 0.000), ('brown', 0.025), ...]`. 

The list should be **hard-coded**, meaning that you should run your permutation tests in your notebook, not in your `.py` file. The list should also be sorted in **increasing order of p-value**. Make sure your p-values are rounded to **3 decimal places**.

Even though there is randomness in the color composition in each bag, this list gives the likelihood that the machines have a systematic, meaningful, difference in how they blend the colors in each bag.

In [75]:
def ordered_colors():
    """
    ordered_colors returns your answer as an ordered
    list from "most different" to "least different" 
    between the two locations. You list should be a 
    hard-coded list, where each element has the 
    form (color, p-value).

    :Example:
    >>> out = ordered_colors()
    >>> len(out) == 5
    True
    >>> colors = {'green', 'orange', 'purple', 'red', 'yellow'}
    >>> set([x[0] for x in out]) == colors
    True
    >>> all([isinstance(x[1], float) for x in out])
    True
    """
    # BEGIN SOLUTION
    L = []
    for color in skittles.columns[:-1]:
        p = pval_color(skittles, col=color)
        L.append((color, p))

    out = sorted(L, key=lambda x:x[1])
    return out

    return out
    # END SOLUTION

In [76]:
# don't change this cell -- it is needed for the tests to work
q5_out = ordered_colors()
q5_colors = {'green', 'orange', 'purple', 'red', 'yellow'}
q5_test_colors = [x[0] for x in q5_out]

In [77]:
q5_out

[('yellow', 0.0),
 ('red', 0.061),
 ('orange', 0.107),
 ('purple', 0.337),
 ('green', 1.0)]

In [78]:
""" # BEGIN TEST CONFIG
hidden: false
points: 0.5
""" # END TEST CONFIG
len(q5_out) == 5

True

In [79]:
""" # BEGIN TEST CONFIG
hidden: false
points: 0.5
""" # END TEST CONFIG
set([x[0] for x in q5_out]) == q5_colors

True

In [80]:
""" # BEGIN TEST CONFIG
hidden: false
points: 0.5
""" # END TEST CONFIG
all([isinstance(x[1], float) for x in q5_out])

True

In [81]:
""" # BEGIN TEST CONFIG
hidden: false
points: 1
failure_message: 'yellow less than orange'
""" # END TEST CONFIG
q5_test_colors.index('orange') > q5_test_colors.index('yellow')

True

In [82]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'orange less than red'
""" # END TEST CONFIG
q5_test_colors.index('orange') > q5_test_colors.index('red')

True

In [83]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'red greater than or equal to green'
""" # END TEST CONFIG
q5_test_colors.index('green') > q5_test_colors.index('red')

True

In [84]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'green less than purple'
""" # END TEST CONFIG
q5_test_colors.index('purple') < q5_test_colors.index('green')

True

In [85]:
""" # BEGIN TEST CONFIG
hidden: false
points: 1
failure_message: 'smallest pval'
""" # END TEST CONFIG
np.isclose(q5_out[0][1], 0.0)

True

In [86]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: '2nd pval'
""" # END TEST CONFIG
np.isclose(q5_out[1][1], 0.059, atol=0.02)

True

In [87]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: '3rd pval'
""" # END TEST CONFIG
np.isclose(q5_out[2][1], 0.095, atol=0.02)

True

In [88]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: '4th pval'
""" # END TEST CONFIG
np.isclose(q5_out[3][1], 0.302, atol=0.05)

True

In [89]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'largest pval'
""" # END TEST CONFIG
np.isclose(q5_out[4][1], 0.998, atol=0.02)

True

### Question 6 – Overall distributions 🏭

Now, suppose you would like to assess whether the two locations make similar amounts of each color overall. That is, suppose we:
* Combine and count up all the Skittles of each color that were made in Yorkville (e.g. 14303 total red skittles, 9091 total green skittles, etc.)
* Combine and count up all the Skittles of each color that were made in Waco.

**Are these distributions of colors similar?** Is the variation among the bags due to each factory making different amounts of each color?

Use a permutation test to assess whether the distribution of colors of Skittles made in Yorkville is statistically significantly different than those made in Waco. Set a significance level of 0.01 and determine whether you can reject a null hypothesis that answers the question above using a permutation test with 1000 trials. For your test statistic, use the **total variation distance (TVD)**.

Refer to [Lecture 7](https://github.com/dsc-courses/dsc80-2022-fa/blob/main/lectures/07-permutation/notebook/lecture.ipynb) to see an example of a [permutation test](https://www.inferentialthinking.com/chapters/12/Comparing_Two_Samples.html) that uses the [TVD](https://inferentialthinking.com/chapters/11/2/Multiple_Categories.html) as the test statistic. Some guidance:

- Our previous permutation tests have compared the median proportion of (say) orange Skittles in Yorkville bags to the median proportion number of orange Skittles in Waco bags. The role of shuffling was to randomly assign bags to Yorkville and Waco.
- In this permutation test, we are **still** shuffling to randomly assign bags to Yorkville and Waco. The only difference is that after we randomly assign each bag to a factory, we will compute the distribution of colors amongst the two factories and find the TVD between those two distributions.

**Your job:** Create a function `same_color_distribution` that takes in no arguments and outputs a hard-coded **tuple** with the p-value and whether you `'Fail to Reject'` or `'Reject'` the null hypothesis.

In [90]:
# BEGIN SOLUTION NO PROMPT
# helper function
def tvd(data):
    sums = data.groupby('Factory').sum()
    distrs = sums.div(sums.sum(axis=1), axis=0)
    tvd = distrs.diff().iloc[-1].abs().sum() / 2
    
    return tvd


# helper function
def permtest(data):
    N = 1000
    results = []
    dg = data.copy()
    for _ in range(N):
        dg['Factory'] = (
            dg['Factory']
            .sample(frac=1, replace=False)
            .reset_index(drop=True)
        )
        results.append(tvd(dg))

    obs = tvd(data)
    pval = (pd.Series(results) >= obs).mean()
    
    return pval
    
    # END SOLUTION

def same_color_distribution():
    """
    same_color_distribution outputs a hard-coded tuple 
    with the p-value and whether you 'Fail to Reject' or 'Reject' 
    the null hypothesis.

    >>> out = same_color_distribution()
    >>> isinstance(out, tuple)
    True
    >>> isinstance(out[0], float)
    True
    >>> out[1] in ['Fail to Reject', 'Reject']
    True
    """
    # BEGIN SOLUTION
    out = (0.008, 'Reject')
    return out
    # END SOLUTION

In [91]:
# don't change this cell -- it is needed for the tests to work
q6_out = same_color_distribution()

In [92]:
""" # BEGIN TEST CONFIG
hidden: false
points: 0.5
failure_message: 'tuple'
""" # END TEST CONFIG
isinstance(q6_out, tuple)

True

In [93]:
""" # BEGIN TEST CONFIG
hidden: false
points: 1
failure_message: 'wrong output type at index 0'
""" # END TEST CONFIG
isinstance(q6_out[0], float)

True

In [94]:
""" # BEGIN TEST CONFIG
hidden: false
points: 1
failure_message: 'wrong output type at index 1'
""" # END TEST CONFIG
q6_out[1] in ['Fail to Reject', 'Reject']

True

In [95]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'reject the null hypothesis'
""" # END TEST CONFIG
cond1 = q6_out[1] == 'Reject'
cond2 = (q6_out[0] > 0.01) and (q6_out[1] == 'Fail to Reject')
cond1 or cond2

True

In [96]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'p-value, approximate within 0.1'
""" # END TEST CONFIG
np.isclose(q6_out[0], 0.005, atol=0.1)

True

In [97]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'p-value, approximate within 0.05'
""" # END TEST CONFIG
np.isclose(q6_out[0], 0.005, atol=0.05)

True

In [98]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'p-value, approximate within 0.01'
""" # END TEST CONFIG
np.isclose(q6_out[0], 0.005, atol=0.01)

True

In [99]:
""" # BEGIN TEST CONFIG
hidden: false
points: 1
failure_message: 'p-value, approximate within 0.25'
""" # END TEST CONFIG
np.isclose(q6_out[0], 0.005, atol=0.25)

True

In [100]:
""" # BEGIN TEST CONFIG
hidden: false
points: 1
failure_message: 'p-value, approximate within 0.5'
""" # END TEST CONFIG
np.isclose(q6_out[0], 0.005, atol=0.5)

True

### Question 7 – Permutation testing vs. hypothesis testing 🧪

In each of the following scenarios, decide  whether  a  permutation test is appropriate to determine if there is a  significant difference between the quantities described. If a permutation test is appropriate, mark `'P'`. Otherwise, mark `'H'`.

Record your answers in the function `perm_vs_hyp` that outputs a list of length 5, containing the values `'P'` and `'H'`.

1. Compare the DSC 80 pass rate between second years and third years who take the class.
2. Compare the proportion of Data Science majors who have completed DSC 80 and the proportion of Data Science minors who have completed DSC 80.
3. Compare the proportion of students who have iPhones to the proportion of students who have Android phones (for simplicity, assume that all students either have an iPhone or an Android).
4. In DSC 80, we ask all students whether they liked DSC 40A or DSC 40B more. Compare the proportion of students who preferred DSC 40A to the proportion who preferred DSC 40B.
5. Compare the attendance rate of classes that play music before class vs. classes that do not play music before class.

***Hint:*** Think about the type of data you would collect in each case, and how you would simulate new data under the null hypothesis.

In [101]:
def perm_vs_hyp():
    """
    Multiple choice response for Question 8.

    >>> out = perm_vs_hyp()
    >>> ans = ['P', 'H']
    >>> len(out) == 5
    True
    >>> set(out) <= set(ans)
    True
    """
    # BEGIN SOLUTION
    '''
    Scenario 1: Permutation test. These are two different groups. 
    We'd have one list of second years and whether or not they pass, e.g. sophomores = [1, 0, 0, 1, 1, 0, 0],
    and another for third years, e.g. juniors = [1, 1, 1, 1, 1, 0, 0, 1].
    Under our null, we randomly assign students to be second or third years and look at the differences
    in pass rates.
    
    Scenario 2: Permutation test. Again, these are two different groups.
    We'd have one list of DSC majors and whether or not they've completed DSC 80, e.g. majors = [1, 0, 0, 0, 1],
    and another for DSC minors, e.g. minors = [0, 0, 0, 1, 1, 1].
    Under our null, we randomly assign students to be majors or minors and look at differences
    in completion rates.
    
    Scenario 3: Hypothesis test. This is just one group – students in one section of one class, for example.
    To simulate data, I essentially flip a coin – heads is iPhone, tails is Android.
    If the question was, do iPhone users score higher than Android users on average, then we could
    run a permutation test, but that's not the question here.
    
    Scenario 4: Hypothesis test. Same as above, but DSC 40A and DSC 40B instead of iPhone and Android.
    
    Scenario 5: Permutation test. Two different groups – classes that play music before class and classes that
    don't. For both groups, we have separate lists of information.
    '''

    out = ['P', 'P', 'H', 'H', 'P']

    return out
    # END SOLUTION

In [102]:
# don't change this cell -- it is needed for the tests to work
q7_out = perm_vs_hyp()

In [103]:
""" # BEGIN TEST CONFIG
hidden: false
points: 1
failure_message: 'output length should be 5'
""" # END TEST CONFIG
len(q7_out) == 5

True

In [104]:
""" # BEGIN TEST CONFIG
hidden: false
points: 1
failure_message: 'output contains answers other than P or H'
""" # END TEST CONFIG
set(q7_out) <= set(['P', 'H'])

True

In [105]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'sub-question 1'
""" # END TEST CONFIG
q7_out[0] == 'P' # Don't just copy the answer!

True

In [106]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'sub-question 2'
""" # END TEST CONFIG
q7_out[1] == 'P' # Don't just copy the answer!

True

In [107]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'sub-question 3'
""" # END TEST CONFIG
q7_out[2] == 'H' # Don't just copy the answer!

True

In [108]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'sub-question 4'
""" # END TEST CONFIG
q7_out[3] == 'H' # Don't just copy the answer!

True

In [109]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'sub-question 5'
""" # END TEST CONFIG
q7_out[4] == 'P' # Don't just copy the answer!

True

## Part 4: Types of Missingness

First, let's recap the different mechanisms of missingness we studied in lecture.

### Missing by Design (MD)
- The missing field is deliberately missing. The missing field is deliberately set to null or not collected (hence, "missing by design").
- The missingness can be exactly predicted when a column will be null, with only knowledge of the other columns using a function of the rows of the dataset.

### Missing Completely at Random (MCAR)
- The missingness of missing value isn't related to the actual, unreported value itself, nor the values in any other fields. The missingness is not systematic.
- The missingness is unconditionally uniform across rows. MCAR doesn't bias the observed data.
- There is no relationship between the missing data and the any of the other data, observed or missing.

### Missing at Random (MAR)
- The missingness of the missing value has nothing to do with the value itself, but may be related to another field.
- The missingness is uniform across rows, perhaps conditional on another column. MAR biases the observed data, but is fixable.
- There is a systematic relationship between the missing values and the observed data (but not the missing values themselves).
- Difference between MD and MAR: If you can *exactly/always* determine missingness using the other columns, the missingness is MD. If there is just some sort of systematic relationship between the missing columns/values and other columns/values that may help us predict missingness, the missingness is MAR.

### Non-Ignorable (NI, aka NMAR)
- The missingness of the missing value is related to the actual, unreported value.
- NI biases the observed data in unobservable ways.
- There is relationship between the propensity of a value to be missing and its value.
- ***Note:*** In lecture, we referred to non-ignorable missingness as "not missing at random (NMAR)".

### Question 8 – After-purchase surveys 🛒

You run a small e-commerce website and send surveys out to customers after they purchase an item from your store. The survey asks whether the customer is satisfied with their purchase ("Yes" or "No"). Below, you are presented with possible datasets, each of which contains a column `'satisfied'` as described above, as well as a `'customer_id'` number corresponding to the customer and an `'item'` column describing the item that the customer purchased. **The column `'satisfied'` is missing data.**

For each of the following datasets, label the column `'satisfied'` as being `'MD'`, `'MCAR'`, `'MAR'`, or `'NI'`.

1. The dataset consists only of the columns `'customer_id'` and `'satisfied'`.
2. The dataset contains the `'customer_id'` of every customer with an account, even if they didn't make a purchase. Also, in this case, you notice everyone who was sent a survey filled it out.
3. The dataset contains a column specifying if the user later returned the item.
4. The dataset contains a column with the serial number for the item purchased.
5. The dataset contains a column with the price of the item purchased.

Record your answers in the function `after_purchase` that outputs a list of length 5, containing the values `'MD'`, `'MCAR'`, `'MAR'`, or `'NI'`. For some questions there may be multiple good answers, but there is generally one answer that is "best". If you are unsure, ask a tutor, but be prepared to provide justification for whichever answer(s) you think might be right.

***Disclaimer:*** We know that this lab has no hidden tests, and so it is possible to just look at the correct answers by running `grader.check`. This is not a good idea – you should really think about all of the questions here, since similar questions will be on the Midterm Exam.

In [110]:
def after_purchase():
    """
    Multiple choice response for question 8

    >>> out = after_purchase()
    >>> ans = ['MD', 'MCAR', 'MAR', 'NI']
    >>> len(out) == 5
    True
    >>> set(out) <= set(ans)
    True
    """
    # BEGIN SOLUTION
    '''
    Scenario 1: The customer ID column is irrelevant. In the absence of any other information,
    our best guess is that customers who were satisfied or felt neutral didn't feel the need to
    leave a review, whereas dissatisfied customers may have been more likely to leave a review.
    As such, the data is not missing at random, i.e. NI, since the missingness depends on the value
    itself.
    
    Scenario 2: Since we can predict whether or not 'Satisfied' is missing – it is missing if and only if
    a customer didn't order anything (i.e. 'item' is empty) – the data is missing by design.
    
    Scenario 3: Customers who returned their items are more likely to be dissatisfied than those
    who didn't return their items, and dissatisfied customers are more likely to not leave a review.
    As such, the missingness depends on the 'Returned?' column, so the data is missing at random.
    
    Scenario 4: The serial number, like the customer ID, is just some random string which is irrelevant.
    This is the same as Scenario 1.
    
    Scenario 5: Similar to Scenario 3. Perhaps the more expensive items are better, leading to customers
    being more satisfied and less likely to leave a review.
    '''
    out = ['NI', 'MD', 'MAR', 'NI', 'MAR']  # MAR for index=3 as well

    return out
    # END SOLUTION

In [111]:
# don't change this cell -- it is needed for the tests to work
q8_out = after_purchase()

In [112]:
""" # BEGIN TEST CONFIG
hidden: false
points: 1
failure_message: 'output length should be 5'
""" # END TEST CONFIG
len(q8_out) == 5

True

In [113]:
""" # BEGIN TEST CONFIG
hidden: false
points: 1
failure_message: 'output contains answers other than the specified options'
""" # END TEST CONFIG
set(q8_out) <= set(['MD', 'MCAR', 'MAR', 'NI'])

True

In [114]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'sub-question 1: reviewers are more likely to review when dissatified'
""" # END TEST CONFIG
q8_out[0] == 'NI'

True

In [115]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'sub-question 1: partial; one column must be NI or MCAR'
""" # END TEST CONFIG
q8_out[0] in ['NI', 'MCAR']

True

In [116]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'sub-question 2: not the optimal answer'
""" # END TEST CONFIG
q8_out[1] == 'MD'

True

In [117]:
""" # BEGIN TEST CONFIG
hidden: true
points: .5
failure_message: 'sub-question 2: partial'
""" # END TEST CONFIG
q8_out[1] in ['MD', 'MAR']

True

In [118]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'sub-question 3: not the optimal answer'
""" # END TEST CONFIG
q8_out[2] == 'MAR'

True

In [119]:
""" # BEGIN TEST CONFIG
hidden: true
points: .5
failure_message: 'sub-question 3: partial'
""" # END TEST CONFIG
q8_out[2] in ['NI', 'MAR']

True

In [120]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'sub-question 4: not the optimal answer, how does having a serial number affect missingness?'
""" # END TEST CONFIG
q8_out[3] in ['NI', 'MAR']

True

In [121]:
""" # BEGIN TEST CONFIG
hidden: true
points: .5
failure_message: 'sub-question 4: partial'
""" # END TEST CONFIG
q8_out[3] in ['NI', 'MAR', 'MCAR']

True

In [122]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'sub-question 5: not the optimal answer'
""" # END TEST CONFIG
q8_out[4] == 'MAR'

True

In [123]:
""" # BEGIN TEST CONFIG
hidden: true
points: .5
failure_message: 'sub-question 5: partial'
""" # END TEST CONFIG
q8_out[4] in ['MAR', 'NI', 'MCAR']

True

### Question 9 – Miscellaneous missingness questions 🕵️

In each of the following scenarios, choose the best answer out of the missingness types: `'MD'`, `'MCAR'`, `'MAR'`, and `'NI'`. Store your answers in a list of length 5, and have the function `multiple_choice` return that list.

1. UCSD has recently adopted GrubHub as the food pre-ordering app for campus restaurants, so you can order your food ahead of time and stop by before your next class. In a DataFrame of GrubHub app orders, which contains information such as `'restaurant'`, `'name'`, `'items'`, and `'total'`, the column `'delivery_address'` is often missing for UCSD students. Which is the most likely missingness mechanism for this column?


2. In a database of student records that records student profile data, such as `'name'`, `'home_address'`, `'ethnicity'`, etc., sometimes the `'middle_name'` column is missing. Which is the most likely missingness mechanism for this column?


3. The UCSD Club Basketball team creates a signup sheet for potential new members. The sheet contains the columns `'full_name'`, `'year'`, `'email'`, `'favorite_sports'`, `'number_of_sports_played'`, and `'sports_previously_played'`. The team president notices that many students left the `'sports_previously_played'` column blank. Which is the most likely missingness mechanism for this column?


4. After the 2022 Sun God Festival, Associated Students sends out a survey to all students about whether their expectations for the 2022 Sun God Festival were met, with all questions being optional. They notice that many students left the "Were you satisfied with the 2022 Sun God Festival?" question blank. Which is the most likely missingness mechanism for answers to this question?


5. UCSD has been using a two-factor authentication system, DUO, since October 16th, 2019. When using DUO, all UCSD accounts are assigned a unique code. UCSD's Service Desk, who maintains DUO, has a database that stores each user's code and their phone number, which users must provide when they sign up for DUO. They notice that many phone numbers are missing. Which is the most likely missingness mechanism for phone numbers?

In [124]:
def multiple_choice():
    """
    Multiple choice response for question 9

    >>> out = multiple_choice()
    >>> ans = ['MD', 'MCAR', 'MAR', 'NI']
    >>> len(out) == 5
    True
    >>> set(out) <= set(ans)
    True
    >>> out[1] in ans
    True
    """
    # BEGIN SOLUTION
    '''
    Scenario 1: The other columns may provide some information 
    as to whether or not a delivery address is missing. For example,
    some restaurants may only provide pickup, and hence won't need to
    note down a delivery address.
    
    Scenario 2: The other columns provide some information as to
    whether or not a middle name will be missing. Some ethnicities are 
    less likely to use middle names, for example. Since the missingness
    could depend on other columns but not on the missing values themselves,
    the correct mechanism is MAR.
    
    Scenario 3: If a student hasn't played any previous sports,
    there is no reason for them to answer this question. It is hence
    missing by design. (You could maybe argue that maybe some people were
    embarassed by the previous sports they played, and hence it is NI.)
    
    Scenario 4: People who were dissatisfied are less likely to leave
    a response. The missingness depends on the missing value itself,
    so it is NI.
    
    Scenario 5: Since everyone is required to enter a phone number when
    signing up for DUO, and the other columns provide no information on
    phone numbers, the most likely explanation is that phone numbers
    were dropped from the dataset at random.
    '''
    out = ['MAR', 'MAR', 'MD', 'NI', 'MCAR']

    return out
    # END SOLUTION

In [125]:
# don't change this cell -- it is needed for the tests to work
q9_out = multiple_choice()

In [126]:
""" # BEGIN TEST CONFIG
hidden: false
points: 1
failure_message: 'output length should be 5'
""" # END TEST CONFIG
len(q9_out) == 5

True

In [127]:
""" # BEGIN TEST CONFIG
hidden: false
points: 1
failure_message: 'output contains answers other than the specified options'
""" # END TEST CONFIG
set(q9_out) <= set(['MD', 'MCAR', 'MAR', 'NI'])

True

In [128]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'sub-question 1'
""" # END TEST CONFIG
q9_out[0] == 'MAR'

True

In [129]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'sub-question 1: partial'
""" # END TEST CONFIG
q9_out[0] in ['MCAR', 'MAR', 'NI']

True

In [130]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'sub-question 2'
""" # END TEST CONFIG
q9_out[1] in ['MAR', 'NI']

True

In [131]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'sub-question 3'
""" # END TEST CONFIG
q9_out[2] in ['MD', 'MAR']

True

In [132]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'sub-question 4'
""" # END TEST CONFIG
q9_out[3] == 'NI'

True

In [133]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'sub-question 4: partial'
""" # END TEST CONFIG
q9_out[3] in ['NI', 'MCAR']

True

In [134]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'sub-question 5'
""" # END TEST CONFIG
q9_out[4] == 'MCAR'

True

In [135]:
""" # BEGIN TEST CONFIG
hidden: true
points: 1
failure_message: 'sub-question 5: partial'
""" # END TEST CONFIG
q9_out[4] in ['MAR', 'NI', 'MCAR']

True

## Congratulations! You're done! 🏁

Submit your `lab.py` file to Gradescope. Note that you only need to submit the `lab.py` file; this notebook should not be uploaded.

Before submitting, you should ensure that all of your work is in the `lab.py` file. You can do this by running the doctests below, which will verify that your work passes the public tests **and** that your work is in the `lab.py` file. Run the cell below; you should see no output.

In [136]:
!python -m doctest lab.py

Traceback (most recent call last):
  File "/nix/store/x0fw0l4d6zwgfdwbpp23iwhm3c6a1hh3-python3-3.9.6/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/nix/store/x0fw0l4d6zwgfdwbpp23iwhm3c6a1hh3-python3-3.9.6/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/nix/store/x0fw0l4d6zwgfdwbpp23iwhm3c6a1hh3-python3-3.9.6/lib/python3.9/doctest.py", line 2793, in <module>
    sys.exit(_test())
  File "/nix/store/x0fw0l4d6zwgfdwbpp23iwhm3c6a1hh3-python3-3.9.6/lib/python3.9/doctest.py", line 2781, in _test
    m = __import__(filename[:-3])
ModuleNotFoundError: No module named 'lab'


In addition, `grader.check_all()` will verify that your work passes the public tests.